### Import

In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
import seaborn as sns

from random import seed

from sklearn.linear_model import SGDRegressor
from sklearn import preprocessing

from sklearn.kernel_approximation import RBFSampler
import os

In [8]:
# Reference: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of Dataframe is {:.3f} MB'.format(start_mem))
    
    for col in tqdm(df.columns):
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif 'datetime' not in col_type.name:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.3f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### Creating Features

In [3]:
# Downloading data (using wget)

file_path="favorita-grocery-sales-forecasting.zip"

if not os.path.exists(file_path):
    !wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/7391/44328/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1593984946&Signature=TZ8WhKQzNyAp%2B8IRIjBE3f9IPhSdR%2B8izTu2DDZLt1ZJS9M5q5pZsNpMGYYOCFwROdvxHPUf%2FIVoPslSOiRMcBdkBhumDs6xiOt9A5dzgUh6QqH3%2BzX%2F%2Be2FVjW2dg3a%2B%2FmqIwQLD7y%2B8gfRP82VlEMdGcxLLbRliMfy2ZK0BlMZgRZJ7%2BNmsdbm3V6Y%2Fk7YnIiDGH3bBopFwLN02mOhiqb96GC4gD813iLV5DRoSzegViOZjddjSBtKeNlFu86bo9oj2cjI%2BQrxQV%2F2I6IU1lKqXxkkdAl0oFzzfNUwlLForPg0nd8GMaYgdlM6Ga1liBl2QFahMYkwJUM6Hvv%2F6w%3D%3D&response-content-disposition=attachment%3B+filename%3Dfavorita-grocery-sales-forecasting.zip" -c -O 'favorita-grocery-sales-forecasting.zip'
else:
    print("File Already Present")

--2020-07-03 01:43:21--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/7391/44328/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1593984946&Signature=TZ8WhKQzNyAp%2B8IRIjBE3f9IPhSdR%2B8izTu2DDZLt1ZJS9M5q5pZsNpMGYYOCFwROdvxHPUf%2FIVoPslSOiRMcBdkBhumDs6xiOt9A5dzgUh6QqH3%2BzX%2F%2Be2FVjW2dg3a%2B%2FmqIwQLD7y%2B8gfRP82VlEMdGcxLLbRliMfy2ZK0BlMZgRZJ7%2BNmsdbm3V6Y%2Fk7YnIiDGH3bBopFwLN02mOhiqb96GC4gD813iLV5DRoSzegViOZjddjSBtKeNlFu86bo9oj2cjI%2BQrxQV%2F2I6IU1lKqXxkkdAl0oFzzfNUwlLForPg0nd8GMaYgdlM6Ga1liBl2QFahMYkwJUM6Hvv%2F6w%3D%3D&response-content-disposition=attachment%3B+filename%3Dfavorita-grocery-sales-forecasting.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.125.128, 74.125.23.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480014675 (458M) [application/zip]
Saving to: ‘favorit

In [4]:
# unzipping favorita-grocery-sales-forecasting.zip

if os.path.exists('favorita-grocery-sales-forecasting.zip'):
    !unzip 'favorita-grocery-sales-forecasting.zip'
    print("File unzipped Successfully")
else:
    print("File Not Present to unzip")

Archive:  favorita-grocery-sales-forecasting.zip
  inflating: holidays_events.csv.7z  
  inflating: items.csv.7z            
  inflating: oil.csv.7z              
  inflating: sample_submission.csv.7z  
  inflating: stores.csv.7z           
  inflating: test.csv.7z             
  inflating: train.csv.7z            
  inflating: transactions.csv.7z     
File unzipped Successfully


In [5]:
#installing 7zip for extracting .7z files
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [6]:
#Extracting .7z files if they are not already extracted.

for file in os.listdir():
    if file[-3:]=='.7z':
        if os.path.exists(file[:-3]):
            print("="*50)
            print("'{}'Extracted File is Already Present".format(file[:-3]))
        elif file=='oil.csv.7z':
            !p7zip -d 'oil.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='train.csv.7z':
            !p7zip -d 'train.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='stores.csv.7z':
            !p7zip -d 'stores.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='transactions.csv.7z':
            !p7zip -d 'transactions.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='items.csv.7z':
            !p7zip -d 'items.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='holidays_events.csv.7z':
            !p7zip -d 'holidays_events.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='test.csv.7z':
            !p7zip -d 'test.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='sample_submission.csv.7z':
            !p7zip -d 'sample_submission.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        print("="*50)
        


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 666528 bytes (651 KiB)

Extracting archive: sample_submission.csv.7z
--
Path = sample_submission.csv.7z
Type = 7z
Physical Size = 666528
Headers Size = 146
Method = LZMA2:24
Solid = -
Blocks = 1

  0%     72% - sample_submission.csv                            Everything is Ok

Size:       40445582
Compressed: 666528
'sample_submission.csv.7z' File Extracted Successfully

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 47409

In [7]:
#Creating features by excecuting Pre_Processing Feature_engineering.py

exec(open('Pre_Processing Feature_engineering.py').read())

# Train Dataset Initial Date 28/6/2017
# 2 weeks

# Validation Dataset Initial Date 26/7/2017
# 1 week

# Test Dataset Initial Date 16/8/2017
# 1 week


Data Pre-Processing ...



Enter the following for Train Data :

Starting Date (Day/Month/Year) --> 28/6/2017
No. of weeks --> 2

Creating Features for data between Dates --> 2017-06-28 - 2017-07-12 (i.e. 2 weeks) 




Saving 'X_train.csv' File ...
Saving 'y_train.csv' File ...

Enter the following for Validation Data :

Starting Date (Day/Month/Year) --> 26/7/2017
No. of weeks --> 1

Creating Features for data between Dates --> 2017-07-26 - 2017-08-02 (i.e. 1 weeks) 




Saving 'X_val.csv' File ...
Saving 'y_val.csv' File ...

Enter the following for Test Data :

Starting Date (Day/Month/Year) --> 16/8/2017
No. of weeks --> 1

Creating Features for data between Dates --> 2017-08-16 - 2017-08-23 (i.e. 1 weeks) 




Saving 'X_test.csv' File ...

Saving 'sales_2017.csv' File ...
Saving 'stores_items.csv' File ...


### Reading Data

In [ ]:
# Reading X_train.csv and reducing memory usage
X_train=pd.read_csv("X_train.csv")
X_train=reduce_mem_usage(X_train)

# Reading y_train.csv and converting into numpy array
y_train = np.array(pd.read_csv( 'y_train.csv'))

Memory usage of Dataframe is 1617.996 MB



Memory usage after optimization is: 351.461 MB
Decreased by 78.3%


In [ ]:
# Reading X_val.csv and reducing memory usage
X_val=pd.read_csv("X_val.csv")
X_val=reduce_mem_usage(X_val)

# Reading y_val.csv and converting into numpy array
y_val = np.array(pd.read_csv( 'y_val.csv'))

Memory usage of Dataframe is 808.998 MB



Memory usage after optimization is: 175.091 MB
Decreased by 78.4%


In [ ]:
# Reading X_test.csv and reducing memory usage
X_test=pd.read_csv("X_test.csv")
X_test=reduce_mem_usage(X_test)


Memory usage of Dataframe is 808.998 MB



Memory usage after optimization is: 175.730 MB
Decreased by 78.3%


In [9]:
# Reading stores_items.csv
stores_items = pd.read_csv('stores_items.csv', index_col=['store_nbr','item_nbr'])

# Reading items.csv and setting index as item_nbr
items = pd.read_csv( 'items.csv' ).set_index("item_nbr")

items = items.reindex( stores_items.index.get_level_values(1) )
items=reduce_mem_usage(items)


Memory usage of Dataframe is 5.112 MB



Memory usage after optimization is: 1.919 MB
Decreased by 62.5%


### Imputing Nan values

In [ ]:
def find_nan_values(df,replace_by=None):
  '''
  Finds column with Nan values in the given dataframe.

  df = Input Dataframe
  replace_by = None (can be given any value to replace Nan in every column)

  returns df

  '''

    if df.isnull().values.any():
        null_columns=df.columns[df.isnull().any()].tolist()
        for column in null_columns:
            total_values=df.shape[0]
            null_values=df[column].isnull().sum()
            print("No. of Nan Values in '{}' column --> {}".format(column,null_values))
            print("{:.2f} % of Total values".format((100 * null_values) / total_values))
            print("\n")
            if replace_by!=None :
                df[column] = df[column].fillna(np.int(replace_by))
        
    else:
        print("No Column has Nan Values")
        
    return df


In [ ]:
# Replacing Nan values with 0
X_train = find_nan_values(X_train,replace_by=0)

No. of Nan Values in '3days_sale_mean(promo)' column --> 296577
88.52 % of Total values


No. of Nan Values in '3days_sale_mean(no_promo)' column --> 17058
5.09 % of Total values


No. of Nan Values in '7days_sale_mean(promo)' column --> 258814
77.25 % of Total values


No. of Nan Values in '7days_sale_mean(no_promo)' column --> 11991
3.58 % of Total values


No. of Nan Values in '14days_sale_mean(promo)' column --> 246115
73.46 % of Total values


No. of Nan Values in '14days_sale_mean(no_promo)' column --> 6088
1.82 % of Total values


No. of Nan Values in '30days_sale_mean(promo)' column --> 229096
68.38 % of Total values


No. of Nan Values in '30days_sale_mean(no_promo)' column --> 633
0.19 % of Total values


No. of Nan Values in '60days_sale_mean(promo)' column --> 198567
59.27 % of Total values


No. of Nan Values in '60days_sale_mean(no_promo)' column --> 178
0.05 % of Total values


No. of Nan Values in '140days_sale_mean(promo)' column --> 164082
48.98 % of Total values


No

In [ ]:
# Replacing Nan values with 0
X_val = find_nan_values(X_val,replace_by=0)

No. of Nan Values in '3days_sale_mean(promo)' column --> 151822
90.63 % of Total values


No. of Nan Values in '3days_sale_mean(no_promo)' column --> 6683
3.99 % of Total values


No. of Nan Values in '7days_sale_mean(promo)' column --> 134526
80.31 % of Total values


No. of Nan Values in '7days_sale_mean(no_promo)' column --> 4907
2.93 % of Total values


No. of Nan Values in '14days_sale_mean(promo)' column --> 127069
75.86 % of Total values


No. of Nan Values in '14days_sale_mean(no_promo)' column --> 2810
1.68 % of Total values


No. of Nan Values in '30days_sale_mean(promo)' column --> 114612
68.42 % of Total values


No. of Nan Values in '30days_sale_mean(no_promo)' column --> 199
0.12 % of Total values


No. of Nan Values in '60days_sale_mean(promo)' column --> 102245
61.04 % of Total values


No. of Nan Values in '60days_sale_mean(no_promo)' column --> 14
0.01 % of Total values


No. of Nan Values in '140days_sale_mean(promo)' column --> 81935
48.91 % of Total values


No. of

### Feature Selection

In [ ]:
# Loading Top 300 Feature Names (got by training random forest)
import pickle
with open('300_filtered_features.pkl','rb') as file:
    filtered_features = pickle.load( file)

Filtering top 300 features


### Defining SGD Regressor with RBF Kernel Approximation

In [ ]:
def train_rbf_model(X_train,y_train,X_val,y_val,eta,alpha,n_days,items,features,X_test=None):
    '''
    Filter features from the Dataset, 
    Standardizes the data, applies RBF feature transform and then 
    Trains 16 different SGDRegressor models for predicting next 16 days sales . 

    Returns --> * val_pred i.e.predicted values of validation data
                * test_pred i.e.predicted values of test data if present
    '''

    val_pred=[]
    test_pred=[]

    #Training 16 different models for predicting next 16 days sales.
    for i in tqdm(range(16)):
 
        # Filtering features
        x_train = X_train[features[i]]
        x_val = X_val[features[i]]

        #Filtering Features from test dataset if it exists.
        try:
            x_test = X_test[features[i]]
        except:
            pass

        #Standardizing features by making the mean zero and scaling to unit variance.
        scaler = preprocessing.StandardScaler().fit(x_train)
        x_train = scaler.transform(x_train)
        x_val=scaler.transform(x_val)
        try:
            x_test = scaler.transform(x_test)
        except:
            pass

        ''' 
        RBF Kernel trick do not scale well to large numbers of training samples or large numbers of features.
        RBFSampler is cheaper to compute, though, making use of larger feature spaces more efficient.
        So Approximating feature map of an RBF kernel by Monte Carlo approximation of its Fourier transform.
        Typically, it maps the original dataset into a higher dimensional space, approximating the kernel.
        '''
        # Transforming data using RBFSampler
        # Using only 500 dimensional feature transform to reduce time complexity.
        rbf = RBFSampler(gamma=gamma, n_components = 500 ,random_state=1)  
        x_train = rbf.fit_transform(x_train)
        x_val = rbf.transform(x_val)
        try:
            x_test = rbf.transform(x_test)
        except:
            pass

        #Training model
        model = SGDRegressor(eta0=eta,alpha=alpha,max_iter =4000 ,verbose=0)
        model.fit(x_train, y_train [:,i], sample_weight = pd.concat([items["perishable"]] * n_days) * 0.25 + 1  )

        # appending results of prediction on validation set.
        val_pred.append(model.predict(x_val))
        # appending results of prediction on test set if it exists.
        try:
            test_pred.append(model.predict(x_test))
        except:
            pass
            
    if type(X_test) != type(None):
        return val_pred,test_pred
    else:
        return val_pred

### Performance Metric


**NWRMSLE** (Normalized Weighted Root Mean Squared Logarithmic Error)

In [ ]:
def calculate_nwrmsle(true,pred,weight):
    ''' 
    Calculates Normalized Weighted Root Mean Squared Logarithmic Error (nwrmsle)

    true = true labels
    pred =  predicted labels
    weight = weights of datapoints

    returns nwrmsle '''

    temp = (true - np.array(pred).transpose())**2
    temp = temp.sum(axis=1) * weight
    nwrmsle = np.sqrt(temp.sum() / weight.sum() / 16)
    return nwrmsle

### HyperParameter Tuning (Using Grid Search)

In [ ]:
''' Training model for every eta , alpha ,gamma triplet i.e. Grid Search'''

n_days=2
gammas = [0.001, 0.01, 0.1]
etas =[0.0001,0.001,0.01,0.1]
alphas = [0.0001, 0.001, 0.01, 0.1]
count=0
mses=[]
scores=[]

for eta in etas:
    for alpha in alphas:
        for gamma in gammas:
            count+=1
            print("="*50)
            print("Training Model No.{} ...\n\n eta0 = {}, alpha = {} , gamma = {} ".format(count,eta,alpha,gamma))

            # Training SGD Regressor (with rbf kernel approximation)
            val_pred = train_rbf_model(X_train,y_train,X_val,y_val,eta,alpha,n_days,items,filtered_features)

            #Calculating mse for validation data
            val_mse = mean_squared_error(y_val, np.array(val_pred).transpose())
            print("="*50)
            print("val_mse --> ",val_mse)

            #Calculating nwrmsle for validation data
            weight = items["perishable"] * 0.25 + 1
            nwrmsle = calculate_nwrmsle(y_val,y_pred,weight)
            print("NWRMSLE --> ",nwrmsle)
            print("="*50)

            # storing nwrmsle for every model in list
            scores.append(nwrmsle)


Training Model No.1 ...

 eta0 = 0.0001, alpha = 0.0001 , gamma = 0.001 



val_mse -->  0.48704053986091755
val_nwrmsle -->  0.7000349780330108
Training Model No.2 ...

 eta0 = 0.0001, alpha = 0.0001 , gamma = 0.01 



val_mse -->  0.7764790154047032
val_nwrmsle -->  0.887490092948468
Training Model No.3 ...

 eta0 = 0.0001, alpha = 0.0001 , gamma = 0.1 



val_mse -->  1.1365648014630576
val_nwrmsle -->  1.07224357200711
Training Model No.4 ...

 eta0 = 0.0001, alpha = 0.001 , gamma = 0.001 



val_mse -->  0.488411499726755
val_nwrmsle -->  0.7010578850953315
Training Model No.5 ...

 eta0 = 0.0001, alpha = 0.001 , gamma = 0.01 


...


val_mse -->  0.7131673064958333
val_nwrmsle -->  0.8498368259935105
Training Model No.45 ...

 eta0 = 0.1, alpha = 0.01 , gamma = 0.1 



val_mse -->  1.13428313734443
val_nwrmsle -->  1.0713637344116538
Training Model No.46 ...

 eta0 = 0.1, alpha = 0.1 , gamma = 0.001 



val_mse -->  0.6755747429207548
val_nwrmsle -->  0.8261506144755063
Training Model No.47 ...

 eta0 = 0.1, alpha = 0.1 , gamma = 0.01 



val_mse -->  0.9523682514320022
val_nwrmsle -->  0.9825396853331658
Training Model No.48 ...

 eta0 = 0.1, alpha = 0.1 , gamma = 0.1 



val_mse -->  1.1375483982423948
val_nwrmsle -->  1.0727152941237228


In [ ]:
# Finding which model has minimum nwrmsle
for i in range(len(scores)):
    if scores[i] == min(scores):
        print("Best Score :",scores[i])
        print("Model no. ",i+1)

Best Score : 0.6162329786968735
Model no.  37


### Observations :

* Model --> **SGD Regressor with RBF kernel Approx.**
* Best Parameters : 
    * *eta0* = **0.1**
    * *alpha* = **0.0001** 
    * *gamma* = **0.001**

* Best Score (NWRMSLE) = **0.6162**